In [ ]:
# Deanonymizer, given CSG Dicoms Anonymizer idtoname.csv file
# v0.1

In [ ]:
# Forcefully autoreload all python modules
%load_ext autoreload
%autoreload 2

In [ ]:
# AUX FUNCTIONS

import os, sys

cur_path = os.path.realpath('.')
sys.path.append(os.path.join(cur_path, 'csg_fileutil_libs'))  # for unidecode and cleanup_name, because it does not support relative paths (yet?)

import re

from csg_fileutil_libs.aux_funcs import save_df_as_csv, _tqdm, merge_two_df, cleanup_name_df


In [ ]:
# PARAMETERS

# Reference database, from which records will be extracted
ref_db = 'idtoname/demographics_shortened.csv'
id_db = 'idtoname/idtoname.csv'


In [ ]:
import pandas as pd

cref = pd.read_csv(ref_db, sep=';')
cref

In [ ]:
cid = pd.read_csv(id_db, sep=';')
cid

In [ ]:
# Merge csv and fmp final diagnoses if name matches
cmerge = merge_two_df(cref, cid, col='name', mode=0, skip_sanity=True)
cmerge

In [ ]:
cmerge.loc[cmerge['name'].isnull(), 'name'] = cmerge['name2']
cmerge

In [ ]:
# Remap names
def df_remap_names(df, dfremap, col='name', col2='name2', keep_nulls=False):
    '''Remap names in a dataframe using a remapping list (ie, a dataframe with two names columns)'''
    def replace_nonnull_df(x, repmap):
        replacement = repmap[x]
        return replacement if replacement is not None else x
    repmap = dfremap.set_index(col2)[col].to_dict()
    df2 = df.copy().reset_index()
    if keep_nulls:
        # Much faster but if there are nulls they will be replaced
        df2[col] = df2[col].map(repmap)
    else:
        # Slower but remap only if the remap is not null
        df2[col] = df2[col].apply(lambda x: replace_nonnull_df(x, repmap))
    return df2

cid2 = df_remap_names(cid, cmerge, 'name', 'name2')
del cid2['index']
cid2

In [ ]:
cref.reset_index(inplace=True)
cref.set_index('name', inplace=True)
cid2.reset_index(inplace=True)
del cid2['index']
cid2.set_index('name', inplace=True)
cid2

In [ ]:
cref['anonid'] = cid2
cref

In [ ]:
save_df_as_csv(cref.reset_index(), 'demo_unanonymized.csv', ['name', 'anonid'], 'name')

In [ ]:
save_df_as_csv(cid2.reset_index(), 'demo_unanomyzed_names_list.csv', ['name'], 'name')